In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from networkx.algorithms.community.centrality import girvan_newman
from collections import Counter

In [2]:
df = pd.read_csv('../../../Data/final_cool_dataset.csv')

In [3]:
coords = pd.read_json('../../../Data/total.json')

In [4]:
coords = coords.T.reset_index()

coords_origin = coords.rename(columns = {'index':'ORIGIN',0:'longitude_origin',1:'latitude_origin'})
coords_dest = coords.rename(columns = {'index':'DEST',0:'longitude_dest',1:'latitude_dest'})

data_filtered_coord = pd.merge(df,coords_origin, on = ['ORIGIN'])

data_filtered_coord = pd.merge(data_filtered_coord,coords_dest, on = ["DEST"])

mask1 = ~data_filtered_coord['ORIGIN'].isin(['SYA','TNK','RBN']) & ~data_filtered_coord['DEST'].isin(['SYA','TNK','RBN'])

coords_name_df = pd.DataFrame()
coords_name_df['ORIGIN'] = data_filtered_coord[mask1]['ORIGIN']
coords_name_df['COORDS'] = list(zip(data_filtered_coord[mask1]['latitude_origin'],data_filtered_coord[mask1]['longitude_origin']))

In [5]:
G = nx.DiGraph()

In [6]:
pos = dict()
for i,j in zip(coords_name_df['ORIGIN'],coords_name_df['COORDS']):
    pos[i] = (j[0],j[1])
    G.add_node(i)

In [7]:
for i,j in zip(data_filtered_coord[mask1]['ORIGIN'],data_filtered_coord[mask1]['DEST']):
    G.add_edge(i,j)

In [8]:
colors = ['red', 'green', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

In [9]:
dictresult = dict()
list_comm = []
for _ in range(10):
    lp_test = list(nx.community.label_propagation.asyn_lpa_communities(G))
    for i in G.nodes():
        for j in range(len(lp_test)):
            if i in lp_test[j]:
                dictresult.setdefault(i,[]).append(j)

In [10]:
df1 = pd.DataFrame.from_dict(dictresult, dtype=object)

In [11]:
df1 = df1.transpose()

In [12]:
df2 = df1.copy()

In [13]:
for i in df2.iterrows():
    for j in range(len(i[1])):
        i[1][j] = colors[i[1][j]]

In [ ]:
fig = plt.figure(figsize =(300,200))
for _ in range(len(df2.columns)):
    col_list = [df2.loc[node][_] for node in G.nodes]
    nx.draw(G,pos = pos,with_labels = True,font_color = 'white',node_size = 5000,node_color = col_list,edge_color = 'lightgray')
    plt.savefig('comm'+ str(_) +'.png')

## TEST

In [ ]:
col_list = [df2.loc[node][0] for node in G.nodes]

In [ ]:
fig = plt.figure(figsize = (300,200))
nx.draw(G,pos = pos,with_labels = True,font_color = 'white',node_size = 5000,node_color = col_list,edge_color = 'lightgray')

## OTHER IDEA

In [ ]:
dict_shared = dict()
for i in range(10):
    for A1 in dictresult.keys():
        for A2 in dictresult.keys():
            if dictresult[A1][i] == dictresult[A2][i]:
                dict_shared.setdefault(A1, []).append(A2)

In [ ]:
dict_final = dict()
for i,j in dict_shared.items():
    dict_final[i] = Counter(j)